# Commercial and residential hourly load profiles

In [2]:
import os
import io
import calendar
import datetime
import threading

import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
DATASET_ROOT = 'https://openei.org/datasets/files/961/pub/'
COMMERCIAL_LOAD_ROOT = DATASET_ROOT + 'COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT/'
RESIDENTIAL_LOAD_ROOT = DATASET_ROOT + 'RESIDENTIAL_LOAD_DATA_E_PLUS_OUTPUT/'

## Commercial load data

In [4]:
def get_commercial_load_locations(ssl_verify=True, proxy_settings=None):
    # Convert root page content to BeautifulSoup
    page = requests.get(COMMERCIAL_LOAD_ROOT, timeout=10, verify=ssl_verify, proxies=proxy_settings)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get the bulleted list of locations
    location_tags = soup.body.ul
    
    # Convert bulleted list to DataFrame
    locations = []

    for link in location_tags.find_all('a')[1:]:
        # [1:] to skip the link to the parent directory
        loc_root = link['href']
        country, state_abbr, name, _ = loc_root.split('_')

        name = ' '.join(name.split('.')[:-1])

        locations.append({'country': country, 'state': state_abbr, 'name': name, 'link': COMMERCIAL_LOAD_ROOT + loc_root})

    df_locations = pd.DataFrame.from_records(locations)
    
    return df_locations
    

In [5]:
df_locations = get_commercial_load_locations(ssl_verify=False)

test_link = df_locations.loc[df_locations['state'] == 'AL'].iloc[0].link

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [6]:
def get_commercial_building_types(location_root, ssl_verify=True, proxy_settings=None):
    # Convert root page content to BeautifulSoup
    page = requests.get(location_root, timeout=10, verify=ssl_verify, proxies=proxy_settings)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get the bulleted list of building types
    building_tags = soup.body.ul
    
    # Convert bulleted list to DataFrame
    building_types = []

    for link in building_tags.find_all('a')[1:]:
        # [1:] to skip the link to the parent directory
        csv_link = link['href']
        name = csv_link.split('_')[0]

        building_types.append({'name': name, 'link': location_root + csv_link})

    df_building_types = pd.DataFrame.from_records(building_types)
    
    return df_building_types

In [7]:
location_root = test_link
df_building_types = get_commercial_building_types(location_root, ssl_verify=False)

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [8]:
csv_link = df_building_types.iloc[0].link

In [9]:
full_link = csv_link

In [10]:
full_link

'https://openei.org/datasets/files/961/pub/COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT/USA_AL_Anniston.Metro.AP.722287_TMY3/RefBldgFullServiceRestaurantNew2004_v1.3_7.1_3A_USA_GA_ATLANTA.csv'

In [11]:
with requests.Session() as req:
    http_request = req.get(full_link,
                            proxies=None, 
                            timeout=6, 
                            verify=False,
                            stream=True)
    if http_request.status_code != requests.codes.ok:
        http_request.raise_for_status()

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [12]:
data_down = http_request.content.decode(http_request.encoding)
csv_data = pd.read_csv(io.StringIO(data_down))

In [13]:
csv_data['Electricity:Facility [kW](Hourly)']

0       22.547756
1       14.688283
2       14.714919
3       14.725921
4       14.881438
5       22.667171
6       37.583041
7       41.885971
8       41.909304
9       34.435673
10      41.961567
11      42.697291
12      42.025939
13      34.502457
14      34.502059
15      34.487671
16      44.037893
17      46.140269
18      46.137539
19      38.775658
20      38.752766
21      38.678435
22      38.566716
23      39.337644
24      22.612165
25      14.761469
26      14.784827
27      14.797915
28      14.949273
29      22.732786
          ...    
8730    46.199237
8731    38.862988
8732    38.845807
8733    38.766622
8734    38.638121
8735    39.392353
8736    22.677490
8737    14.832474
8738    14.857125
8739    14.867916
8740    15.026982
8741    22.813719
8742    37.728043
8743    42.015316
8744    42.018109
8745    34.547051
8746    42.093042
8747    42.780887
8748    42.152174
8749    34.617042
8750    34.614627
8751    34.593529
8752    44.152418
8753    46.250598
8754    46

In [14]:
df_building_types

,link,name
0,https://openei.org/datasets/files/961/pub/COMM...,RefBldgFullServiceRestaurantNew2004
1,https://openei.org/datasets/files/961/pub/COMM...,RefBldgHospitalNew2004
2,https://openei.org/datasets/files/961/pub/COMM...,RefBldgLargeHotelNew2004
3,https://openei.org/datasets/files/961/pub/COMM...,RefBldgLargeOfficeNew2004
4,https://openei.org/datasets/files/961/pub/COMM...,RefBldgMediumOfficeNew2004
5,https://openei.org/datasets/files/961/pub/COMM...,RefBldgMidriseApartmentNew2004
6,https://openei.org/datasets/files/961/pub/COMM...,RefBldgOutPatientNew2004
7,https://openei.org/datasets/files/961/pub/COMM...,RefBldgPrimarySchoolNew2004
8,https://openei.org/datasets/files/961/pub/COMM...,RefBldgQuickServiceRestaurantNew2004
9,https://openei.org/datasets/files/961/pub/COMM...,RefBldgSecondarySchoolNew2004


## Residential 

In [15]:
# Convert root page content to BeautifulSoup
page = requests.get(RESIDENTIAL_LOAD_ROOT, timeout=10, verify=False, proxies=None)
soup = BeautifulSoup(page.content, 'html.parser')
    
# Get the bulleted list of load types
load_tags = soup.body.ul
   
# Convert bulleted list to DataFrame
load_types = []

for link in load_tags.find_all('a')[1:]:
    # [1:] to skip the link to the parent directory
    locations_root = link['href']

    load_types.append({'name': locations_root[:-1], 'link': RESIDENTIAL_LOAD_ROOT + locations_root})

df_load_types = pd.DataFrame.from_records(load_types)

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [16]:
test_link = df_load_types.iloc[0].link

In [30]:
# Convert root page content to BeautifulSoup
page = requests.get(test_link, timeout=10, verify=False, proxies=None)
soup = BeautifulSoup(page.content, 'html.parser')
    
location_tags = soup.body.ul

# Convert bulleted list to DataFrame
locations = []

for link in location_tags.find_all('a')[1:]:
    # [1:] to skip the link to the parent directory
    csv_link = link['href']
    print(csv_link)
    
    _, _, name, _, _ = csv_link.split('_')
    name = ' '.join(name.split('.')[:-1])

    locations.append({'name': name, 'link': test_link + csv_link})

df_locations = pd.DataFrame.from_records(locations)

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


USA_AK_Anchorage.Intl.AP.702730_TMY3_BASE.csv
USA_AK_Fairbanks.Intl.AP.702610_TMY3_BASE.csv
USA_AL_Anniston.Metro.AP.722287_TMY3_BASE.csv
USA_AL_Auburn-Opelika.AP.722284_TMY3_BASE.csv
USA_AL_Birmingham.Muni.AP.722280_TMY3_BASE.csv
USA_AL_Dothan.Muni.AP.722268_TMY3_BASE.csv
USA_AL_Fort.Rucker-Cairns.Field.722269_TMY3_BASE.csv
USA_AL_Gadsen.Muni.AWOS.722285_TMY3_BASE.csv
USA_AL_Huntsville.Intl.AP-Jones.Field.723230_TMY3_BASE.csv
USA_AL_Maxwell.AFB.722265_TMY3_BASE.csv
USA_AL_Mobile-Downtown.AP.722235_TMY3_BASE.csv
USA_AL_Mobile-Rgnl.AP.722230_TMY3_BASE.csv
USA_AL_Montgomery-Dannelly.Field.722260_TMY3_BASE.csv
USA_AL_Muscle.Shoals.Rgnl.AP.723235_TMY3_BASE.csv
USA_AL_Troy.Air.Field.722267_TMY3_BASE.csv
USA_AL_Tuscaloosa.Muni.AP.722286_TMY3_BASE.csv
USA_AR_Batesville.AWOS.723448_TMY3_BASE.csv
USA_AR_Bentonville.AWOS.723444_TMY3_BASE.csv
USA_AR_El.Dorado-Goodwin.Field.723419_TMY3_BASE.csv
USA_AR_Fayetteville-Drake.Field.723445_TMY3_BASE.csv
USA_AR_Flippin.AWOS.723447_TMY3_BASE.csv
USA_AR_For

In [29]:
link['href']

'USA_WY_Worland.Muni.AP.726665_TMY3_BASE.csv'

In [19]:
full_link = df_locations.iloc[0].link

with requests.Session() as req:
    http_request = req.get(full_link,
                            proxies=None, 
                            timeout=6, 
                            verify=False,
                            stream=True)
    if http_request.status_code != requests.codes.ok:
        http_request.raise_for_status()

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [20]:
data_down = http_request.content.decode(http_request.encoding)
csv_data = pd.read_csv(io.StringIO(data_down))

In [21]:
csv_data['Electricity:Facility [kW](Hourly)']

0       1.019364
1       0.887116
2       0.833463
3       0.818020
4       0.812916
5       0.865135
6       1.087779
7       1.429298
8       1.382221
9       1.220633
10      1.271489
11      1.291682
12      1.215100
13      1.169484
14      1.169849
15      1.252507
16      1.525574
17      2.067353
18      2.371010
19      2.320927
20      2.170542
21      2.013591
22      1.694778
23      1.375976
24      1.058186
25      0.935044
26      0.898032
27      0.888657
28      0.886075
29      0.938923
          ...   
8730    2.304117
8731    2.143907
8732    2.050409
8733    1.881184
8734    1.551405
8735    1.240520
8736    0.922002
8737    0.803810
8738    0.757236
8739    0.749157
8740    0.755866
8741    0.832948
8742    1.061529
8743    1.363395
8744    1.344902
8745    1.200316
8746    1.198637
8747    1.266138
8748    1.202129
8749    1.167227
8750    1.172657
8751    1.257689
8752    1.568955
8753    2.123067
8754    2.399626
8755    2.191529
8756    2.079864
8757    1.9126

In [22]:
df_locations = get_commercial_load_locations(ssl_verify=False)
df_locations

C:\Users\rconcep\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,country,link,name,state
0,USA,https://openei.org/datasets/files/961/pub/COMM...,Anchorage Intl AP,AK
1,USA,https://openei.org/datasets/files/961/pub/COMM...,Fairbanks Intl AP,AK
2,USA,https://openei.org/datasets/files/961/pub/COMM...,Anniston Metro AP,AL
3,USA,https://openei.org/datasets/files/961/pub/COMM...,Auburn-Opelika AP,AL
4,USA,https://openei.org/datasets/files/961/pub/COMM...,Birmingham Muni AP,AL
5,USA,https://openei.org/datasets/files/961/pub/COMM...,Dothan Muni AP,AL
6,USA,https://openei.org/datasets/files/961/pub/COMM...,Fort Rucker-Cairns Field,AL
7,USA,https://openei.org/datasets/files/961/pub/COMM...,Gadsen Muni AWOS,AL
8,USA,https://openei.org/datasets/files/961/pub/COMM...,Huntsville Intl AP-Jones Field,AL
9,USA,https://openei.org/datasets/files/961/pub/COMM...,Maxwell AFB,AL


In [23]:
df_locations.state.unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
       'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)